In [7]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.utils import save_image
from tqdm import tqdm
from PIL import Image

# --- Simpele UNet Generator ---

# Deze eenvoudige UNet heeft:
# - Een eerste conv-laag die 6 inputkanalen (3 noisy + 3 condition) omzet naar 64 feature maps.
# - Een downsampling-laag (conv met stride 2) die de resolutie halveert en het aantal feature maps verdubbelt.
# - Een upsampling-laag (transposed conv) die teruggaat naar de oorspronkelijke resolutie.
# - Een skip-verbinding die de output van de eerste laag toevoegt aan de upsampled features.
# - Een output-conv-laag die de voorspelde ruis (3 kanalen) genereert.
class SimpleUNet(nn.Module):
    def __init__(self, in_channels=6, out_channels=3, base_channels=64):
        super(SimpleUNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, base_channels, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU(inplace=True)
        self.down = nn.Conv2d(base_channels, base_channels * 2, kernel_size=4, stride=2, padding=1)
        self.up = nn.ConvTranspose2d(base_channels * 2, base_channels, kernel_size=4, stride=2, padding=1)
        self.out_conv = nn.Conv2d(base_channels, out_channels, kernel_size=3, stride=1, padding=1)

    def forward(self, x, time_embedding=None):
        # Eerste conv-laag
        x1 = self.relu(self.conv1(x))
        # Downsampling
        x2 = self.relu(self.down(x1))
        # Upsampling
        x3 = self.relu(self.up(x2))
        # Skip-verbinding: voeg de features van conv1 toe
        x3 = x3 + x1
        # Voorspel de ruis (output)
        out = self.out_conv(x3)
        return out

# --- DDPM Noise Scheduler en Sampler ---

class DDPM(nn.Module):
    def __init__(self, model, timesteps=1000, beta_start=1e-4, beta_end=0.02, device="cuda"):
        super(DDPM, self).__init__()
        self.device = torch.device(device if torch.cuda.is_available() else "cpu")
        self.model = model.to(self.device)
        self.timesteps = timesteps
        self.betas = torch.linspace(beta_start, beta_end, timesteps, device=self.device)
        self.alphas = 1.0 - self.betas
        self.alpha_cumprod = torch.cumprod(self.alphas, dim=0)
        self.alphas = self.alphas.to(self.device)
        self.alpha_cumprod = self.alpha_cumprod.to(self.device)
        
    def add_noise(self, x, t):
        """
        Voeg ruis toe aan x volgens het DDPM schema op timestep t.
        x: clean (high quality) image, t: tensor met timesteps per sample.
        """
        noise = torch.randn_like(x)
        sqrt_alpha_cumprod = torch.sqrt(self.alpha_cumprod[t]).view(-1, 1, 1, 1)
        sqrt_one_minus_alpha_cumprod = torch.sqrt(1 - self.alpha_cumprod[t]).view(-1, 1, 1, 1)
        noisy = sqrt_alpha_cumprod * x + sqrt_one_minus_alpha_cumprod * noise
        return noisy, noise

    def forward(self, x, cond, t):
        """
        Forward pass tijdens training: voorspel de toegevoegde ruis.
        x: noisy image (afgeleid van de high quality image)
        cond: condition (low quality image)
        t: timestep (per sample)
        """
        x_in = torch.cat([x, cond], dim=1)
        return self.model(x_in, t)

    def p_sample(self, x, t, cond):
        """
        Voer één reverse diffusion stap uit.
        x: huidige x_t (batch van afbeeldingen)
        t: huidige timestep (gehele getal)
        cond: condition (low quality image)
        """
        t_tensor = torch.tensor([t], device=self.device).long().expand(x.size(0))
        model_out = self.model(torch.cat([x, cond], dim=1), t_tensor)
        alpha_cumprod_t = self.alpha_cumprod[t]
        sqrt_recip_alpha_t = 1.0 / torch.sqrt(self.alphas[t])
        sqrt_one_minus_alpha_cumprod_t = torch.sqrt(1 - alpha_cumprod_t)
        x0_pred = (x - sqrt_one_minus_alpha_cumprod_t * model_out) * sqrt_recip_alpha_t
        if t == 0:
            return x0_pred
        beta_t = self.betas[t]
        alpha_cumprod_prev = self.alpha_cumprod[t-1]
        posterior_variance = beta_t * (1 - alpha_cumprod_prev) / (1 - alpha_cumprod_t)
        noise = torch.randn_like(x)
        mean = torch.sqrt(alpha_cumprod_prev) * x0_pred
        x_prev = mean + torch.sqrt(posterior_variance) * noise
        return x_prev

    def sample(self, cond):
        """
        Voer het volledige reverse diffusieproces uit om een herstelde afbeelding (x0) te genereren,
        gegeven de condition (low quality image).
        """
        x = torch.randn_like(cond).to(self.device)
        for t in reversed(range(1, self.timesteps)):
            x = self.p_sample(x, t, cond)
        t0 = 0
        t_tensor = torch.tensor([t0], device=self.device).long().expand(x.size(0))
        model_out = self.model(torch.cat([x, cond], dim=1), t_tensor)
        sqrt_recip_alpha0 = 1.0 / torch.sqrt(self.alphas[t0])
        sqrt_one_minus_alpha_cumprod0 = torch.sqrt(1 - self.alpha_cumprod[t0])
        x0_pred = (x - sqrt_one_minus_alpha_cumprod0 * model_out) * sqrt_recip_alpha0
        return x0_pred

# --- Dataset ---

class ImageDataset(Dataset):
    def __init__(self, lq_dir, hq_dir, transform=None):
        self.lq_paths = sorted([os.path.join(lq_dir, f) for f in os.listdir(lq_dir) 
                                  if f.lower().endswith((".png", ".jpg", ".jpeg"))])
        self.hq_paths = sorted([os.path.join(hq_dir, f) for f in os.listdir(hq_dir) 
                                  if f.lower().endswith((".png", ".jpg", ".jpeg"))])
        if len(self.lq_paths) == 0 or len(self.hq_paths) == 0:
            raise ValueError(f"Dataset is leeg! Controleer paden: {lq_dir} en {hq_dir}")
        self.transform = transform

    def __len__(self):
        return len(self.lq_paths)

    def __getitem__(self, index):
        lq_image = self._load_image(self.lq_paths[index])
        hq_image = self._load_image(self.hq_paths[index])
        if self.transform:
            lq_image = self.transform(lq_image)
            hq_image = self.transform(hq_image)
        return lq_image, hq_image

    def _load_image(self, path):
        return Image.open(path).convert("RGB")

# --- Training en Test Framework ---

class ImageRestorationModel:
    def __init__(self, lq_dir, hq_dir, test_lq_dir, test_hq_dir, device="cuda"):
        self.device = torch.device(device if torch.cuda.is_available() else "cpu")
        # Let op: de SimpleUNet krijgt 6 inputkanalen (3 noisy + 3 condition) en voorspelt 3 outputkanalen (ruis)
        self.model = SimpleUNet(in_channels=6, out_channels=3).to(self.device)
        self.ddpm = DDPM(self.model, timesteps=1000, device=device).to(self.device)
        self.optimizer = optim.Adam(self.model.parameters(), lr=1e-4)
        self.criterion = nn.MSELoss()

        transform = transforms.Compose([transforms.ToTensor()])
        self.train_dataset = ImageDataset(lq_dir, hq_dir, transform=transform)
        self.test_dataset = ImageDataset(test_lq_dir, test_hq_dir, transform=transform)
        self.train_loader = DataLoader(self.train_dataset, batch_size=8, shuffle=True)
        self.test_loader = DataLoader(self.test_dataset, batch_size=1, shuffle=False)

    def train(self, epochs=10):
        self.model.train()
        for epoch in range(epochs):
            epoch_loss = 0
            for lq, hq in tqdm(self.train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
                lq, hq = lq.to(self.device), hq.to(self.device)
                # Kies voor elke sample een willekeurige timestep t
                t = torch.randint(0, self.ddpm.timesteps, (lq.size(0),), device=self.device)
                # Voeg ruis toe aan de high quality image
                noisy_hq, noise = self.ddpm.add_noise(hq, t)
                # Geef de noisy image samen met de low quality image (condition) aan het model
                pred_noise = self.ddpm(noisy_hq, lq, t)
                loss = self.criterion(pred_noise, noise)
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                epoch_loss += loss.item()
            print(f"Epoch {epoch+1}: Gemiddelde loss = {epoch_loss/len(self.train_loader):.6f}")


# --- Main ---

if __name__ == "__main__":
    # Stel de paden in voor de Low Quality (LQ) en High Quality (HQ) afbeeldingen
    lq_dir = "data/train/low"
    hq_dir = "data/train/high"
    test_lq_dir = "data/test/low"
    test_hq_dir = "data/test/high"

    # Initialiseer en train het model
    model = ImageRestorationModel(lq_dir, hq_dir, test_lq_dir, test_hq_dir, device="cuda")
    model.train(epochs=50)



Epoch 1/50: 100%|█████████████████████████████████████████████████████████████████████| 270/270 [00:09<00:00, 27.39it/s]


Epoch 1: Gemiddelde loss = 0.362906


Epoch 2/50: 100%|█████████████████████████████████████████████████████████████████████| 270/270 [00:09<00:00, 27.48it/s]


Epoch 2: Gemiddelde loss = 0.117131


Epoch 3/50: 100%|█████████████████████████████████████████████████████████████████████| 270/270 [00:09<00:00, 27.82it/s]


...

Epoch 48: Gemiddelde loss = 0.025812


Epoch 49/50: 100%|████████████████████████████████████████████████████████████████████| 270/270 [00:09<00:00, 28.05it/s]


Epoch 49: Gemiddelde loss = 0.021099


Epoch 50/50: 100%|████████████████████████████████████████████████████████████████████| 270/270 [00:09<00:00, 27.42it/s]

Epoch 50: Gemiddelde loss = 0.028148


In [11]:
import torch

# Sla alleen de UNet-gewichten op
torch.save(model.model.state_dict(), "unet_weights.pth")

# Als je ook de optimizer wilt opslaan (bijvoorbeeld om later de training te hervatten):
checkpoint = {
    "model_state_dict": model.model.state_dict(),
    "optimizer_state_dict": model.optimizer.state_dict(),
    # Voeg hier eventueel andere relevante informatie toe, zoals het huidige epoch-nummer.
}
torch.save(checkpoint, "checkpoint.pth")

print("Model en optimizer succesvol opgeslagen.")


Model en optimizer succesvol opgeslagen.


In [8]:
import torch
from torchvision.utils import save_image

# Aantal test samples dat je in het overzicht wilt opnemen
n_samples = 10

# Zorg dat je model in evaluatiemodus staat
model.model.eval()
device = model.device
comparisons = []

with torch.no_grad():
    for idx, (lq, hq) in enumerate(model.test_loader):
        if idx >= n_samples:
            break

        lq = lq.to(device)
        hq = hq.to(device)

        # Genereer de herstelde afbeelding via het reverse diffusionproces
        restored = model.ddpm.sample(lq)
        
        # Verwijder de batch-dimensie (aangenomen dat batch_size == 1 is)
        lq_img = lq.squeeze(0).cpu()
        restored_img = restored.squeeze(0).cpu()
        hq_img = hq.squeeze(0).cpu()
        
        # Combineer de afbeeldingen horizontaal: [low quality | restored | ground truth]
        combined = torch.cat([lq_img, restored_img, hq_img], dim=2)
        comparisons.append(combined)

# Plak alle rijen (elk een test sample) verticaal aan elkaar
overview = torch.cat(comparisons, dim=1)
save_image(overview, "overview.png")
print("Overview opgeslagen als 'overview.png'")


Overview opgeslagen als 'overview.png'


In [10]:
import torch
from PIL import Image
import torchvision.transforms as transforms
from torchvision.utils import save_image

# Geef het pad op naar jouw JPG-afbeelding
image_path = "test.jpg"  # pas dit pad aan

# Laad de afbeelding en converteer naar RGB
img = Image.open(image_path).convert("RGB")

# Gebruik dezelfde transformatie als tijdens training (bijv. ToTensor)
transform = transforms.ToTensor()
img_tensor = transform(img).unsqueeze(0)  # Voeg een batch-dimensie toe

# Verplaats de tensor naar het device (bijv. cuda of cpu) dat in je model gebruikt wordt
img_tensor = img_tensor.to(model.device)

# Zorg dat het model in evaluatiemodus staat
model.model.eval()

# Voer inference uit: de input fungeert als condition voor het reverse diffusieproces
with torch.no_grad():
    restored_img = model.ddpm.sample(img_tensor)

# Sla de gerestoreerde afbeelding op als JPG
save_image(restored_img, "restored_single.jpg")
print("Afbeelding opgeslagen als 'restored_single.jpg'")


Afbeelding opgeslagen als 'restored_single.jpg'
